<a href="https://colab.research.google.com/github/clustering-jun/GNU-MachineLearning/blob/main/L11-3_%EB%AA%A8%EB%8D%B8_%ED%8F%89%EA%B0%80_%EB%B0%8F_%EC%84%A0%ED%83%9D%EC%9D%98_%ED%99%9C%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **분류 문제 사례: 신용 카드 사기 감지 데이터**

In [39]:
import pandas as pd

df = pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [41]:
df.isnull().sum().max()

0

In [42]:
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [43]:
from sklearn.preprocessing import RobustScaler

rob_scaler = RobustScaler()

df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1, 1))
df['scaled_time'] = rob_scaler.fit_transform(df['Time'].values.reshape(-1, 1))

df.drop(['Time', 'Amount'], axis = 1, inplace = True)

In [44]:
scaled_amount = df['scaled_amount']
scaled_time = df['scaled_time']

df.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
df.insert(0, 'scaled_amount', scaled_amount)
df.insert(1, 'scaled_time', scaled_time)

df.head()


,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,1.783274,-0.994983,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.269825,-0.994983,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,4.983721,-0.994972,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,1.418291,-0.994972,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,0.670579,-0.994960,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


In [46]:
df = df.sample(frac=1)

fraud_df = df.loc[df['Class'] == 1]
non_fraud_df = df.loc[df['Class'] == 0][:492]

normal_distributed_df = pd.concat([fraud_df, non_fraud_df])

new_df = normal_distributed_df.sample(frac=1, random_state=0)

new_df.head()

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
150667,0.782505,0.107708,-10.850282,6.727466,-16.760583,8.425832,-10.252697,-4.192171,-14.077086,7.168288,-3.683242,-15.239962,8.030708,-16.060306,0.270530,-14.952981,-0.241095,-11.866731,-15.486990,-5.748652,4.130031,-0.646818,2.541637,0.135535,-1.023967,0.406265,0.106593,-0.026232,-1.464630,-0.411682,1
149207,-0.295955,0.071876,0.409213,0.725981,-0.700465,-0.061315,1.356819,1.742069,-0.513043,-1.329903,1.130053,-0.441198,2.221402,-1.447188,1.728105,2.337098,0.416409,-1.176964,1.066436,-0.417489,-0.443847,-0.482465,1.784001,0.661814,-0.089167,-0.747922,0.150673,0.058741,0.335389,0.300420,0
172787,1.713407,0.429352,-2.628922,2.275636,-3.745369,1.226948,-1.132966,-1.256353,-1.752420,0.281736,-1.792343,-4.820779,4.176098,-4.619010,0.742690,-7.636963,-0.197888,-4.140396,-7.604715,-1.524505,1.271322,0.133106,0.870730,1.269473,-0.265494,-0.480549,0.169665,0.096081,0.070036,0.063768,1
122550,-0.203451,-0.095043,-0.410972,1.038562,1.383014,0.096705,-0.019522,-0.705667,0.615072,0.079943,-0.323723,-0.541109,0.189197,-0.237606,-0.615946,-0.250377,1.194136,0.026847,0.442462,-0.611750,-0.568945,0.037145,-0.220233,-0.568600,0.071719,0.330452,-0.297674,0.083443,0.256986,0.096972,0
28507,1.061273,-0.583336,1.170080,-0.195091,-0.112889,-0.039350,-0.203927,-0.639160,0.183047,-0.229145,0.127582,-0.184113,-0.904961,0.084477,0.514547,0.200050,1.101891,0.523972,-0.621291,-0.393146,0.368202,0.163183,-0.228588,-0.869646,-0.071178,-0.394827,0.246103,0.844895,-0.102545,0.014207,0


In [47]:
print(new_df['Class'].value_counts()/len(new_df))

Class
1    0.5
0    0.5
Name: count, dtype: float64


In [49]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    new_df.drop('Class', axis=1),
    new_df['Class'],
    test_size = 0.2,
    random_state = 2023
)

X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [50]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

models = {"Decision Tree": DecisionTreeClassifier(),
          "Random Forest": RandomForestClassifier(),
          "KNN": KNeighborsClassifier(),
          "Logisitic Regression": LogisticRegression(),
          "SVM": SVC()}

from sklearn.model_selection import cross_val_score

for key, model in models.items():
    model.fit(X_train, y_train)
    training_score = cross_val_score(model, X_train, y_train, cv=5)
    print(model.__class__.__name__, "Training Score:", round(training_score.mean(), 3))

DecisionTreeClassifier Training Score: 0.896
RandomForestClassifier Training Score: 0.939
KNeighborsClassifier Training Score: 0.921
LogisticRegression Training Score: 0.933
SVC Training Score: 0.922


In [53]:
from sklearn.model_selection import GridSearchCV

DT_params = {"criterion": ["gini", "entropy"],
             "max_depth": list(range(2, 4, 1)),
             "min_samples_leaf": list(range(5, 7, 1))}
DT_grid = GridSearchCV(DecisionTreeClassifier(), DT_params)
DT_grid.fit(X_train, y_train)

RF_params = {"criterion": ["gini", "entropy"],
             "n_estimators": [100, 200, 500, 1000],
             "max_depth": list(range(2, 4, 1))}
RF_grid = GridSearchCV(RandomForestClassifier(), RF_params)
RF_grid.fit(X_train, y_train)

KNN_params = {"n_neighbors": list(range(2, 5, 1)),
              "algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute']}
KNN_grid = GridSearchCV(KNeighborsClassifier(), KNN_params)
KNN_grid.fit(X_train, y_train)

LR_params = {"penalty": ['l1', 'l2'],
             "C": [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
LR_grid = GridSearchCV(LogisticRegression(), LR_params)
LR_grid.fit(X_train, y_train)

SVM_params = {"C": [0.5, 0.7, 0.9, 1],
              "kernel": ['rbf', 'poly', 'sigmoid', 'linear']}
SVM_grid = GridSearchCV(SVC(), SVM_params)
SVM_grid.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.5, 0.7, 0.9, 1],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']})

In [54]:
DT_score = cross_val_score(DT_grid.best_estimator_, X_train, y_train, cv=5)
print('Decision Tree Cross Validation Score:', round(DT_score.mean(), 3))

RF_score = cross_val_score(RF_grid.best_estimator_, X_train, y_train, cv=5)
print('Random Forest Cross Validation Score:', round(RF_score.mean(), 3))

KNN_score = cross_val_score(KNN_grid.best_estimator_, X_train, y_train, cv=5)
print('KNN Cross Validation Score:', round(KNN_score.mean(), 3))

LR_score = cross_val_score(LR_grid.best_estimator_, X_train, y_train, cv=5)
print('Logistic Regression Cross Validation Score:', round(LR_score.mean(), 3))

SVM_score = cross_val_score(SVM_grid.best_estimator_, X_train, y_train, cv=5)
print('SVM Cross Validation Score:', round(SVM_score.mean(), 3))

Decision Tree Cross Validation Score: 0.924
Random Forest Cross Validation Score: 0.931
KNN Cross Validation Score: 0.934
Logistic Regression Cross Validation Score: 0.935
SVM Cross Validation Score: 0.933


In [55]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

LR_pred = LR_grid.best_estimator_.predict(X_test)

test_cm = confusion_matrix(y_test, LR_pred)
test_acc = accuracy_score(y_test, LR_pred)
test_prc = precision_score(y_test, LR_pred)
test_rcll = recall_score(y_test, LR_pred)
test_f1 = f1_score(y_test, LR_pred)

print(test_cm)
print()
print("정확도는 {}%".format(round(test_acc*100, 2)))
print("정밀도는 {}%".format(round(test_prc*100, 2)))
print("재현율은 {}%".format(round(test_rcll*100, 2)))
print("F1는 {}%".format(round(test_f1*100, 2)))

[[88  4]
 [ 9 96]]

정확도는 93.4%
정밀도는 96.0%
재현율은 91.43%
F1는 93.66%


### **연습 문제**
- `bodyPerformance.csv`의 `class`변수에 대해 적절한 과정을 거쳐서 가장 최적이라고 판단되는 분류 모델을 훈련한 뒤 성능평가지표를 출력해보시오.
